# Part 02
----
아래에서는 인구이동 데이터의 1차적인 전처리 과정을 행한다. 우선적으로 읍면동 부분을 제외한 행자부코드를 이용하여 하나의 지역코드로 묶는다. 이후 불필요한 칼럼을 없앤다.

2012년과 2013년의 데이터의 경우 이동 총인구가 마침표로 잘못 기입된 데이터가 있어 이를 수정하고 정수타입으로 변환한다.

또한 누락된 데이터 외의 모든 데이터는 정수타입으로 표기될 수 있으므로 nan값을 -1로 수정 한뒤 정수타입으로 변환시킨다.

세대주 성별의 경우 주민등록상 등록된 번호로 구분이 되어 있기에 일괄적으로 대응되는 1 또는 2로 변환한다.

세대주 구분에서 0과 -1은 모두 비세대주로 간주하여 0으로 통일한다.

마지막으로 데이터 탐색 단계에서 이동 총인구가 -1이거나나 0으로 기입된 데이터 포인트가 119개 있다는 것을 알수 있었으며, 이 경우 이동 남인구와 이동 여인구의 데이터가 모두 0으로 되어있는 것을 감안, 그리고 전체 데이터 크기인 약 6300만과 비교하여 상당히 적은 수인 것도 감안하여 이동 총인구가 0보다 작거나 같은 경우 누락시키기로 한다.

위의 전처리 과정을 위한 데이터 탐구 과정(이동 총인구 데이터의 타입, 세대주 성별 구분 코드, 세대주 관계 코드의 비일관성 발견 등)은 아래 코드에는 생략한다.

In [1]:
%matplotlib inline
import glob

In [2]:
path = 'raw_data\\'

In [3]:
population_columns = ['in_state', 'in_city', 'in_nbr', 'year', 'month', 'in_date',
                      'out_state', 'out_city', 'out_nbr', 'reason',
                      'hh_head', 'head_age', 'head_male', 'mov_multi',
                      'mov_ppl', 'mov_male', 'mov_female']

# we drop move the following columns:
#     date: won't be used as it is too specific
#     nbr: same reason as above
#     mov_multi, mov_ppl: can be derived from mov_male and mov_female
used_columns = ['in_state', 'in_city', 'year', 'month',
                'out_state', 'out_city', 'reason',
                'hh_head', 'head_age', 'head_male',
                'mov_male', 'mov_female']

In [4]:
population_files = glob.glob(path + 'population_*.txt')

In [5]:
# take a random file to fiddle around with
df = pd.read_csv(population_files[np.random.randint(len(population_files))],
                 names=population_columns, usecols=used_columns)

## Data Scrubbing

In [6]:
df.head()

,in_state,in_city,year,month,out_state,out_city,reason,hh_head,head_age,head_male,mov_male,mov_female
0,11,110,2012,1,11,110,9,1.0,64.0,2.0,1,1
1,11,110,2012,1,11,350,1,1.0,45.0,2.0,2,1
2,11,110,2012,1,26,470,1,NaN,NaN,NaN,0,1
3,11,110,2012,1,41,285,6,1.0,66.0,1.0,1,0
4,11,110,2012,1,11,110,9,1.0,69.0,1.0,2,2


In [7]:
# checking null values
df.isnull().any()

in_state      False
in_city       False
year          False
month         False
out_state     False
out_city      False
reason        False
hh_head        True
head_age       True
head_male      True
mov_male      False
mov_female    False
dtype: bool

In [8]:
# replace all null values to -1
df = df.replace(to_replace=np.NaN, value=-1)

In [9]:
# we convert everything into int
df = df.astype(int)

In [10]:
# create a column with combined address codes
df['in_address'] = df.in_state * 1000 + df.in_city
df['out_address'] = df.out_state * 1000 + df.out_city
df = df.drop(['in_state', 'in_city', 'out_state', 'out_city'], axis=1)

In [11]:
df.head()

,year,month,reason,hh_head,head_age,head_male,mov_male,mov_female,in_address,out_address
0,2012,1,9,1,64,2,1,1,11110,11110
1,2012,1,1,1,45,2,2,1,11110,11350
2,2012,1,1,-1,-1,-1,0,1,11110,26470
3,2012,1,6,1,66,1,1,0,11110,41285
4,2012,1,9,1,69,1,2,2,11110,11110


In [12]:
# anomalies below were discovered much later in the process
# and are spread out through different files
# we leave these here for the sake of completeness

In [13]:
# change gender codes to 1 (male) or 0 (female) accordingly
# this is due to how gender specifying code works in korean id system
df.loc[(df.head_male == 9) | (df.head_male == 3), 'head_male'] = 1
df.loc[df.head_male != 1, 'head_male'] = 0

In [14]:
# change the missing data values -1 (originally null) to 0 for consistency
# there are cases where values were simply missing rather than being marked as 0
df.loc[df.hh_head == -1, 'hh_head'] = 0

In [15]:
# final check
df.sample(5)

,year,month,reason,hh_head,head_age,head_male,mov_male,mov_female,in_address,out_address
555318,2012,2,9,1,54,1,1,0,11440,41630
1103771,2012,7,9,1,29,0,0,1,11710,11230
1742930,2012,6,9,1,69,1,1,1,27290,27230
4672328,2012,1,2,0,-1,0,0,1,44770,30170
4959414,2012,9,2,0,-1,0,1,0,46130,46130


## Data Aggregation

In [16]:
# make a new column indicating whether the movement happend within the are or not
df['foreign'] = (df.in_address != df.out_address) * 1

# we derive these two columns from others
df['mov_ppl'] = df.mov_male + df.mov_female
df['mov_multi'] = (df.mov_ppl > 1) * 1
df['house_total'] = 1

In [17]:
# there are rare occasions where the data of number of people moved is missing
# there are exactly 116 entries of this kind, and as we have more than 63,000,000
# data points, we simply remove them
df = df.loc[df.mov_ppl > 0]

In [18]:
df.sample(5)

,year,month,reason,hh_head,head_age,head_male,mov_male,mov_female,in_address,out_address,foreign,mov_ppl,mov_multi,house_total
1742853,2012,6,3,1,46,1,1,4,27290,27290,0,5,1,1
958606,2012,10,2,0,-1,0,0,1,11650,11170,1,1,0,1
67126,2012,9,2,0,-1,0,0,1,11170,11170,0,1,0,1
2360501,2012,4,9,0,-1,0,1,0,30140,11710,1,1,0,1
4602428,2012,3,2,1,38,0,1,2,44210,44210,0,3,1,1


In [19]:
# defining a custom function to use when aggregating dataframes
# when calculating the average of the ages, we only count those greater than or equal to 0
# as missing values are indicated by -1
def age_average(series):
    if series[series >= 0].count() != 0:
        return series[series >= 0].sum() / series[series >= 0].count()
    else:
        return -1

In [20]:
# this is a dictionary that will be used to aggregate the original data
agg_functions = {
    'hh_head': 'sum',
    'head_age': age_average,
    'head_male': 'sum',
    'mov_multi': 'sum',
    'mov_ppl': 'sum',
    'mov_male': 'sum',
    'house_total': 'count',
}

In [21]:
# we create two dataframes, one for people moving into area codes and
# another for people moving out of area codes
# this way, although each data point gets 'duplicated', we do this so that we may
# compute population changes for area codes (and other stuff) just in case if they become needed
df_i = df.groupby(['year', 'month', 'in_address', 'foreign', 'reason'], as_index=False).agg(agg_functions)
df_o = df.groupby(['year', 'month', 'out_address', 'foreign', 'reason'], as_index=False).agg(agg_functions)

In [22]:
# add in a new column to indicate whether the data is for people moving in or out
df_i['move_in'] = 1
df_o['move_in'] = 0

In [23]:
df_i.sample(5)

,year,month,in_address,foreign,reason,hh_head,head_age,head_male,mov_multi,mov_ppl,mov_male,house_total,move_in
38968,2012,12,41465,1,9,347,43.023055,238,196,874,432,477,1
28069,2012,9,29140,1,9,133,39.390977,75,52,319,162,228,1
30156,2012,9,47730,0,9,7,47.000000,4,4,21,13,13,1
30504,2012,9,48820,0,2,8,44.000000,6,7,47,23,34,1
12941,2012,4,46890,1,6,7,64.000000,1,2,11,5,7,1


In [24]:
df_o.sample(5)

,year,month,out_address,foreign,reason,hh_head,head_age,head_male,mov_multi,mov_ppl,mov_male,house_total,move_in
23666,2012,7,48123,1,5,7,41.285714,5,0,12,9,12,0
34265,2012,10,50130,0,2,26,41.807692,17,24,153,73,119,0
20288,2012,6,48129,1,5,3,55.666667,1,0,4,1,4,0
34606,2012,11,11710,1,1,669,38.732436,483,212,1184,675,786,0
20777,2012,7,11530,1,1,388,38.435567,274,103,656,380,456,0


In [25]:
io_rename_dic = {'in_address': 'area_code', 'out_address': 'area_code'}

In [26]:
# rename the columns for consistency to support concatenation later
df_i = df_i.rename(columns=io_rename_dic)
df_o = df_o.rename(columns=io_rename_dic)

In [27]:
# note that mov_female column is dropped since we can derive it from other columns
io_column_order = ['year', 'month', 'area_code', 'move_in', 'foreign',
                   'house_total', 'hh_head', 'head_age', 'head_male', 'mov_multi',
                   'mov_ppl', 'mov_male']

In [28]:
# make sure columns are in the same order just in case
df_i = df_i[io_column_order]
df_o = df_o[io_column_order]
df = pd.concat([df_i, df_o])

In [29]:
# check the final result
df.sample(5)

,year,month,area_code,move_in,foreign,house_total,hh_head,head_age,head_male,mov_multi,mov_ppl,mov_male
21644,2012,7,41250,1,1,99,29,41.275862,22,29,150,76
23446,2012,7,47930,1,0,31,28,39.142857,21,9,53,35
30712,2012,9,48820,0,0,51,49,47.469388,25,24,96,44
4395,2012,2,31110,0,0,9,2,41.500000,1,1,10,5
5938,2012,2,46230,1,0,146,105,41.923810,67,51,244,139


## Automation

In [30]:
# in the following section below, we combine the codes above
# and turn them into functions so that we may automate the whole process

In [31]:
# takes a population migration data and apply scrubbing
def pop_scrub(df):
    df = df.replace(to_replace=np.NaN, value=-1)
    df = df.astype(int)
    
    df['in_address'] = df.in_state * 1000 + df.in_city
    df['out_address'] = df.out_state * 1000 + df.out_city
    df = df.drop(['in_state', 'in_city', 'out_state', 'out_city'], axis=1)
    
    df.loc[(df['head_male'] == 9) | (df['head_male'] == 3), 'head_male'] = 1
    df.loc[df.head_male != 1, 'head_male'] = 0
    df.loc[df.hh_head == -1, 'hh_head'] = 0
    
    return df

In [32]:
# groups the migration data by year, month, area code, migrate, and reason
def pop_agg(df):
    df['foreign'] = (df.in_address != df.out_address) * 1
    df['mov_ppl'] = df.mov_male + df.mov_female
    df['mov_multi'] = (df.mov_ppl > 1) * 1
    df['house_total'] = 1
    df = df.loc[df.mov_ppl > 0]
    
    df_i = df.groupby(['year', 'month', 'in_address', 'foreign', 'reason'], as_index=False).agg(agg_functions)
    df_o = df.groupby(['year', 'month', 'out_address', 'foreign', 'reason'], as_index=False).agg(agg_functions)
    
    df_i['move_in'] = 1
    df_o['move_in'] = 0
    df_i = df_i.rename(columns=io_rename_dic)
    df_o = df_o.rename(columns=io_rename_dic)
    df_i = df_i[io_column_order]
    df_o = df_o[io_column_order]
    
    return pd.concat([df_i, df_o])

In [33]:
df = []
for population_file in population_files:
    df_temp = pd.read_csv(population_file, names=population_columns, usecols=used_columns)
    
    df_temp = pop_scrub(df_temp)
    df_temp = pop_agg(df_temp)
    
    df.append(df_temp)
    # print statements to check progress
    print(population_file[-8:-4], end=' ')

2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 

In [34]:
pd.concat(df).to_csv(path + '02_pop.csv', index=False)